In [3]:
import torch

ModuleNotFoundError: No module named 'torch'

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GACSE(nn.Module):
    def __init__(self, n_user, n_item, norm_adj, args):
        super(GACSE, self).__init__()
        self.n_user = n_user
        self.n_item = n_item
        self.device = args.device
        self.emb_size = args.embed_size
        self.batch_size = args.batch_size
        self.node_dropout = args.node_dropout[0]
        self.mess_dropout = args.mess_dropout
        self.batch_size = args.batch_size

        # *************************************************
        self.weighted_sample_num = args.weighted_sample_num
        self.n_heads = 3
        self.neighbors_num = [1] + eval(args.neighbors_num)
        self.node_dropout_flag = args.node_dropout_flag
        #**************************************************

        self.norm_adj = norm_adj

        self.layers = eval(args.layer_size)
        self.decay = eval(args.regs)[0]

        """
        *********************************************************
        Init the weight of user-item.
        """
        self.embedding_dict, self.weight_dict = self.init_weight()
        # print("embedding_dict",self.embedding_dict, "weight_dict",self.weight_dict)

        """
        *********************************************************
        Get sparse adj.
        """
        self.sparse_norm_adj = self._convert_sp_mat_to_sp_tensor(self.norm_adj)

        # self.ua_embeddings, self.ia_embeddings = self._trans_all_embed(self.embedding_dict['user_emb'], self.embedding_dict['item_emb'])
        # print("initial ua_embeddings", self.ua_embeddings, "initial ia_embeddings", self.ia_embeddings)

    def init_weight(self):
        # xavier init
        initializer = nn.init.xavier_uniform_

        embedding_dict = nn.ParameterDict({
            'user_emb': nn.Parameter(initializer(torch.empty(self.n_user,
                                                 self.emb_size))),
            'item_emb': nn.Parameter(initializer(torch.empty(self.n_item,
                                                 self.emb_size))),
            'cse_user_emb': nn.Parameter(initializer(torch.empty(self.n_user,
                                                 self.emb_size))),
            'cse_item_emb': nn.Parameter(initializer(torch.empty(self.n_item,
                                                 self.emb_size)))
        })
        
        tar=np.load(lgcn_data1+'ttarr.npy')
        war=np.load(lgcn_data1+'woarr.npy')
#         #embedding_user.weight.data.copy_(torch.from_numpy(tar))
#         #embedding_item.weight.data.copy_(torch.from_numpy(war))         
#         embedding_dict = nn.ParameterDict({
#             'user_emb': embedding_user,
#             'item_emb': embedding_item 
#         })
        
#         #embedding_dict['user_emb'].update.weight.data.copy_(torch.from_numpy(tar))
        embedding_dict['user_emb'].data = torch.FloatTensor(tar)
        embedding_dict['item_emb'].data = torch.FloatTensor(war)
        embedding_dict['user_emb'].requires_grad = False
        embedding_dict['item_emb'].requires_grad = False
        
        weight_dict = nn.ParameterDict()
        self.layers_list = [self.emb_size] + self.layers
        
        weight_dict.update({'W_trans': nn.Parameter(initializer(torch.empty(self.emb_size,
                                                                    self.layers_list[0])))})
        weight_dict.update({'b_trans': nn.Parameter(initializer(torch.empty(1, self.layers_list[0])))})

        weight_dict.update({'W_att': nn.Parameter(initializer(torch.empty(self.emb_size*2,
                                                                    self.layers_list[0])))})
        weight_dict.update({'V_att': nn.Parameter(initializer(torch.empty(self.layers_list[0],1)))})

        weight_dict.update({'W_att_l': nn.Parameter(initializer(torch.empty(self.layers_list[0],
                                                                    self.layers_list[0])))})
        weight_dict.update({'W_att_r': nn.Parameter(initializer(torch.empty(self.layers_list[0],
                                                                    self.layers_list[0])))})                                                            

        return embedding_dict, weight_dict

    def _convert_sp_mat_to_sp_tensor(self, X):
        coo = X.tocoo()
        i = torch.LongTensor([coo.row, coo.col])
        v = torch.from_numpy(coo.data).float()
        return torch.sparse.FloatTensor(i, v, coo.shape)

    def sparse_dropout(self, x, rate, noise_shape):
        random_tensor = 1 - rate
        random_tensor += torch.rand(noise_shape).to(x.device)
        dropout_mask = torch.floor(random_tensor).type(torch.bool)
        i = x._indices()
        v = x._values()

        i = i[:, dropout_mask]
        v = v[dropout_mask]

        out = torch.sparse.FloatTensor(i, v, x.shape).to(x.device)
        return out * (1. / (1 - rate))

    def create_bpr_loss(self, users, pos_items, neg_items):
        pos_scores = torch.sum(torch.mul(users, pos_items), axis=1)
        neg_scores = torch.sum(torch.mul(users, neg_items), axis=1)
        pos_neg_scores = torch.sum(torch.mul(pos_items, neg_items), axis=1)
#         maxi = nn.LogSigmoid()(pos_scores - neg_scores)

#         mf_loss = -1 * torch.mean(maxi)
#         print(pos_neg_scores)

        mf_loss = torch.mean(nn.Softplus()((-1*(pos_scores - neg_scores  - nn.ReLU()(pos_neg_scores)))))

        # cul regularizer
        regularizer = (torch.norm(users) ** 2
                       + torch.norm(pos_items) ** 2
                       + torch.norm(neg_items) ** 2) / 2
        emb_loss = self.decay * regularizer / self.batch_size

        return mf_loss + emb_loss, mf_loss, emb_loss

    def rating(self, u_g_embeddings, pos_i_g_embeddings):
        return torch.matmul(u_g_embeddings, pos_i_g_embeddings.t())

    def forward(self, users, pos_items_u, neg_items_u, users_pos, users_neg, pos_items_pos, pos_items_neg, neg_items_pos, neg_items_neg, ua_embeddings, ia_embeddings):

        u_g_embeddings = []
        pos_i_g_embeddings = []
        neg_i_g_embeddings = []
        for i in range(len(self.neighbors_num) - 1):
            u_x, u_neib = ua_embeddings[users[i], :], ia_embeddings[users[i+1], :]
            pos_iu_x, pos_iu_neib = ia_embeddings[pos_items_u[i], :], ua_embeddings[pos_items_u[i+1], :]
            
            u_g = self._agg(u_x, u_neib, self.neighbors_num[i+1])
            # print("u_g",u_g.device)
            pos_i_g = self._agg(pos_iu_x, pos_iu_neib, self.neighbors_num[i+1])
            # print("pos_i_g",pos_i_g.device)
            u_g_embeddings.append(u_g)
            pos_i_g_embeddings.append(pos_i_g)
            if len(neg_items_u)>0:
                neg_iu_x, neg_iu_neib = ia_embeddings[neg_items_u[i], :], ua_embeddings[neg_items_u[i+1], :]
                neg_i_g = self._agg(neg_iu_x, neg_iu_neib, self.neighbors_num[i+1]) 
                # print("neg_i_g",neg_i_g.device)
                
                neg_i_g_embeddings.append(neg_i_g)


        u_cse = self.embedding_dict['user_emb'][users[0], :]
        u_p_cse = self.embedding_dict['cse_user_emb'][users_pos, :]
        u_n_cse = self.embedding_dict['cse_user_emb'][users_neg, :]

        pos_i_cse = self.embedding_dict['item_emb'][pos_items_u[0], :]
        pos_i_pos_cse = self.embedding_dict['cse_item_emb'][pos_items_pos, :]
        pos_i_neg_cse = self.embedding_dict['cse_item_emb'][pos_items_neg, :]

        u_g_embeddings += [u_cse]
        pos_i_g_embeddings += [pos_i_cse]
        u_g_embeddings = torch.cat(u_g_embeddings, 1)
        pos_i_g_embeddings = torch.cat(pos_i_g_embeddings, 1)
        
        if len(neg_items_u)>0:
            neg_i_cse = self.embedding_dict['item_emb'][neg_items_u[0], :]
            neg_i_pos_cse = self.embedding_dict['cse_item_emb'][neg_items_pos, :]
            neg_i_neg_cse = self.embedding_dict['cse_item_emb'][neg_items_neg, :]
            neg_i_g_embeddings += [neg_i_cse]
            neg_i_g_embeddings = torch.cat(neg_i_g_embeddings, 1)
            return u_g_embeddings, pos_i_g_embeddings, neg_i_g_embeddings, u_cse, u_p_cse, u_n_cse, pos_i_cse, pos_i_pos_cse, pos_i_neg_cse, neg_i_cse, neg_i_pos_cse, neg_i_neg_cse
        
        else:
            return u_g_embeddings, pos_i_g_embeddings,  torch.empty(1),  torch.empty(1),  torch.empty(1),  torch.empty(1),  torch.empty(1),  torch.empty(1),  torch.empty(1),  torch.empty(1),  torch.empty(1),  torch.empty(1)
        
    def _trans_all_embed(self):
        A_hat = self.sparse_dropout(self.sparse_norm_adj,
                                    self.node_dropout,
                                    self.sparse_norm_adj._nnz()) if self.node_dropout_flag else self.sparse_norm_adj
        # print("A_hat",A_hat.device)
        local_embeddings = torch.cat([self.embedding_dict['user_emb'],
                                    self.embedding_dict['item_emb']], 0).to(self.device)
        # print("local_embeddings",local_embeddings.device)
        
        message_embeddings = torch.sparse.mm(A_hat, local_embeddings)
        # print("message_embeddings",message_embeddings.device)
        # print("message_embeddings",self.weight_dict['W_trans'].device, self.weight_dict['b_trans'].device)
        message_embeddings = nn.LeakyReLU(negative_slope=0.2)(torch.matmul(message_embeddings, self.weight_dict['W_trans'].to(self.device)) \
                                             + self.weight_dict['b_trans'].to(self.device))
        
        users_embedings = message_embeddings[:self.n_user, :]
        item_embeddings = message_embeddings[self.n_user:, :]

        return users_embedings, item_embeddings


    def _agg(self, x, neighbors, num_neighbors):
        x_att = torch.reshape(x, (-1, 1, self.layers_list[0]))
#         print(x_att.shape)
        neighbors_att = torch.reshape(neighbors, (-1, num_neighbors, self.layers_list[0]))
#         print(neighbors_att.shape)
        # neighbors_att = tf.transpose(neighbors_att, perm=[0,2,1])
        x_att = x_att.repeat(1, num_neighbors,1)
#         x_att = torch.repeat_interleave(x_att, torch.tensor([1, num_neighbors,1]), dim=1)
#         print(x_att.shape)
        all_emb = torch.cat([x_att, neighbors_att], 2)
        all_emb = torch.reshape(all_emb, (-1, self.layers_list[0]*2))

        scores = nn.Tanh()(torch.matmul(all_emb, self.weight_dict['W_att'].to(self.device)))
        scores = torch.matmul(scores, self.weight_dict['V_att'].to(self.device))
        scores = torch.reshape(scores, (-1, 1, num_neighbors))
        scores = nn.Softmax(dim=2)(scores)

        # neighbors_att = tf.transpose(neighbors_att, perm=[0,2,1])
        agg_neib = torch.squeeze(torch.matmul(scores, neighbors_att))

        # out = tf.nn.leaky_relu(agg_neib + tf.multiply(x, agg_neib))
        out_l = nn.LeakyReLU(negative_slope=0.2)(torch.matmul(x + agg_neib, self.weight_dict['W_att_l'].to(self.device)))
        out_r = nn.LeakyReLU(negative_slope=0.2)(torch.matmul(torch.mul(x, agg_neib), self.weight_dict['W_att_r'].to(self.device)))
        # out = tf.matmul(out, self.weights['W_att_trans']) + self.weights['b_att_trans']
        out = out_l + out_r
        return F.normalize(out,  p=2, dim=1)
    
    
    def create_point_wise_loss(self, entity, pos_entity, neg_entity):
        epsilon = 1e-7
        # print('entity', entity.device)
        # print('pos_entity', pos_entity.device)
        # print('neg_entity', neg_entity.device)
        
        entity = torch.reshape(entity, (-1, 1, self.emb_size))
        
        # print("entity.shape",entity.shape)
        pos_entity = torch.reshape(pos_entity, (-1, self.weighted_sample_num, self.emb_size))
        
        
        # print("pos_entity.shape",pos_entity.shape)
        pos_entity = torch.transpose(pos_entity, 2,1)
        # print("pos_entity.shape",pos_entity.shape)
        pos_scores = torch.squeeze(torch.sum(torch.clamp(torch.log(nn.Sigmoid()(torch.matmul(entity, pos_entity)) + epsilon), -10, 0), 2))
        # print("pos_scores.shape",pos_scores.shape)
        neg_entity = torch.reshape(neg_entity, (-1, self.weighted_sample_num, self.emb_size))
        neg_entity = torch.transpose(neg_entity, 2,1)
        neg_scores = torch.squeeze(torch.sum(torch.clamp(torch.log(nn.Sigmoid()(torch.matmul(entity, neg_entity)) + epsilon), -10, 0), 2))
        # neg_scores = tf.log(tf.nn.sigmoid(tf.reduce_sum(tf.multiply(entity, neg_entity), axis=1)) + epsilon)
        # print('neg_scores', neg_scores.device)
        # print('pos_scores', pos_scores.device)
        return (-1*(torch.mean(pos_scores - neg_scores)))